In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value = b_init(shape=(units, ), dtype="float32"), trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [4]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.02378732  0.02853826 -0.11968675  0.05676496]
 [ 0.02378732  0.02853826 -0.11968675  0.05676496]], shape=(2, 4), dtype=float32)


In [5]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

In [6]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape = (input_dim, units), initializer = "random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units, ), initializer="zeros", trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2,))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

In [13]:
# 레이어는 훈련 불가능한 가중치를 가질 수 있다.
# assign_add : 기존의 값에서 인수값을 더함
# reduce_sum : axis 를 기준으로 값을 더함. 차원이 축소됨.
class ComputeSum(keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value = tf.zeros((input_dim,)), trainable=False)
    
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total

x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]


In [18]:
print("weights: ", len(my_sum.weights))
print("non-trainable weights:", len(my_sum.non_trainable_weights))

print("trainable_weights:", my_sum.trainable_weights)

weights:  1
non-trainable weights: 1
trainable_weights: []


In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape = (input_shape[-1], self.units),
            initializer = "random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,) , initializer="random_normal", trainable=True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

linear_layer = Linear(32)

y = linear_layer(x)

In [19]:
# 재귀적으로 구성 가능한 레이어

class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)
    
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))
print("weights:", len(mlp.weights))
print("trainable weight:", len(mlp.trainable_weights))

weights: 6
trainable weight: 6


In [20]:
# add_loss() 메서드
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate
    
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

In [21]:
class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)
    
    def call(self, inputs):
        return self.activity_reg(inputs)

layer = OuterLayer()
assert len(layer.losses) == 0

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1

In [23]:
# 가중치에 패널티를 주는 이유는 오버피팅을 막기 위해서. 
# kernel_regularizer : 각 층별 파라미터 or 출력값에 패널티를 부여. 그것에 대한 주요 효과는 오버피팅 방지. 

class OuterLayerWithKernelRegularizer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayerWithKernelRegularizer, self).__init__()
        self.dense = keras.layers.Dense(
            32, kernel_regularizer=tf.keras.regularizers.l2(1e-3)
        )
    
    def call(self, inputs):
        return self.dense(inputs)

layer = OuterLayerWithKernelRegularizer()
_ = layer(tf.zeros((1, 1)))

print(layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0020382036>]


In [24]:
# from_logits : 모델이 출력하는 output이 logit인지 아닌지를 판단하는 것. 
# from_logits=True이고 만일 2개의 분류라면, 결과값이 [0, 1] 이 아닌 [0.1, 0.9] 이런식으로 "확률" 로 나오게됨
"""
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

for x_batch_train, y_batch_train in train_dataset:
    with tf.GradientTape()as tape:
        logits = layer(x_batch_train)
        loss_value = loss_fn(y_batch_train, logits)
        loss_value += sum(model.losses)
    
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

"""

NameError: name 'train_dataset' is not defined

In [27]:
import numpy as np

inputs = keras.Input(shape=(3, ))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

1/1 [==============================] - 0s 46ms/step - loss: 0.0199


In [30]:
# add_metric()
# 여기서 logits는 예측값(pred)을 의미. target은 label
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()
    
    def call(self, targets, logits, sample_weights=None):
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)
        
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")
        
        return tf.nn.softmax(logits)

In [36]:
layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))

layer.metrics: [<tensorflow.python.keras.metrics.BinaryAccuracy object at 0x7ff8d9418820>]
current accuracy value: 1.0


In [42]:
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

1/1 [==============================] - 0s 229ms/step - loss: 0.9191 - binary_accuracy: 0.0000e+00


In [43]:
# 레이어에서 선택적으로 직렬화를 활성화할 수 있다.
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        return {"units": self.units}
    
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'units': 64}


In [44]:
# 초기화시 부모 클래스에 **kwargs 전달하기
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units, ), initializer="random_normal", trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config

    
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'name': 'linear_6', 'trainable': True, 'dtype': 'float32', 'units': 64}


In [45]:
class CustomDropout(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate
    
    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

In [49]:
# layer : layer or block으로 불림
# model : model, network으로 불림.
# fit(), save() 등을 호출해야하는 경우라면 Model을 사용하고 그렇지 않은 경우라면 Layer를 사용하자. 

class ResNet(tf.keras.Model):
    
    def __init__(self):
        super(ResNet, self).__init__()
        self.block_1 = ResNetBlock()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)
    
    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)

resnet = ResNet()
dataset = ...
resnet.fit(dataset, epochs=10)
resnet.save(filepath)

NameError: name 'ResNetBlock' is not defined

지금까지 배운 것 정리
- Layer는 상태(__init__(), build()) 및 일부 계산(call())을 캡슐화한다.
- 레이어를 재귀적으로 중첩하여 새롭고 더 큰 계산 블록을 만들 수 있다.
- 레이어는 add_loss(), add_metric()을 통해 메트릭뿐만 아니라 손실을 생성 및 추적할 수 있다. 
- 훈련하려는 외부 컨테이너는 Model이다. Model은 Layer와 비슷하지만, 훈련 및 직렬화 유틸리티가 추가되었다. 

In [74]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    
class Encoder(layers.Layer):
    
    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()
    
    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z

class Decoder(layers.Layer):
    
    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")
    
    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)

class VariationalAutoEncoder(keras.Model):
    
    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [75]:
# Mnist 훈련셋으로 작성해보자.

original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)


epochs = 2

# GradientTape : tf 2.0의 자동미분 기능. 자동 미분을 통해 동적으로 Gradient 값들을 확인해 볼 수 있다는 장점. 
for epoch in range(epochs):
    print(f"Start of epoch {epoch}")
    
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            print(reconstructed)
            # loss 계산하기
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)

        # 자동 미분한 값.
        grads = tape.gradient(loss, vae.trainable_weights)
        
        # 오차 역전파 - weight 업데이트
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        
        loss_metric(loss)
        
        if step % 100 == 0:
            print(f"step {step}: mean loss = {loss_metric.result():.4f}")
    

Start of epoch 0
tf.Tensor(
[[0.43198273 0.53926325 0.41804683 ... 0.44642615 0.5071694  0.58154047]
 [0.34491563 0.37025565 0.49377084 ... 0.42748773 0.46966588 0.49509022]
 [0.3812223  0.49734446 0.49654627 ... 0.5069141  0.44490802 0.5271926 ]
 ...
 [0.39885926 0.48308352 0.47132194 ... 0.4332949  0.5211846  0.46213692]
 [0.43373668 0.42846504 0.520784   ... 0.4102093  0.5024845  0.5764835 ]
 [0.51846504 0.49455482 0.5531564  ... 0.40880558 0.4995705  0.60292864]], shape=(64, 784), dtype=float32)
step 0: mean loss = 0.3270
tf.Tensor(
[[0.34548348 0.45542234 0.49778092 ... 0.5453984  0.43787524 0.5355891 ]
 [0.38591364 0.47996294 0.41595474 ... 0.44235212 0.50544745 0.47060737]
 [0.46727324 0.46435145 0.48273945 ... 0.49790028 0.45729205 0.5048195 ]
 ...
 [0.39377487 0.43628132 0.45956165 ... 0.4483252  0.48820668 0.5675961 ]
 [0.3844879  0.49516046 0.43987137 ... 0.4450549  0.52914417 0.5168865 ]
 [0.39200583 0.4078884  0.57771945 ... 0.4309301  0.45331898 0.52905196]], shape=(64, 7

tf.Tensor(
[[0.24793422 0.25509518 0.25025263 ... 0.27093202 0.27773067 0.26303262]
 [0.3189836  0.3871209  0.4153673  ... 0.36646053 0.35583624 0.3576386 ]
 [0.3648281  0.35981315 0.31464744 ... 0.366203   0.34915704 0.37686196]
 ...
 [0.26479203 0.4011957  0.35440475 ... 0.38517871 0.32139304 0.34862378]
 [0.39390266 0.33688363 0.32018864 ... 0.38576886 0.44317287 0.34850746]
 [0.2554896  0.3694689  0.36100435 ... 0.2918399  0.34231514 0.36995828]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.3070829  0.43088382 0.33429584 ... 0.36010057 0.42439142 0.41081965]
 [0.29374778 0.41330716 0.3458702  ... 0.31433475 0.34439117 0.34884477]
 [0.2656741  0.3009115  0.32629538 ... 0.23676053 0.28177965 0.28191644]
 ...
 [0.35556898 0.3684768  0.2836946  ... 0.27609754 0.27616054 0.2525503 ]
 [0.33494043 0.30411068 0.2953337  ... 0.3221689  0.3213727  0.2640192 ]
 [0.21148956 0.40779218 0.32433897 ... 0.28369206 0.23498422 0.33558422]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.2900769  

tf.Tensor(
[[0.1596139  0.23429647 0.23562774 ... 0.163414   0.17531723 0.2549065 ]
 [0.2043662  0.21738487 0.23555055 ... 0.1589545  0.19815254 0.22653854]
 [0.1464087  0.20507646 0.22032437 ... 0.2070649  0.20486411 0.23554492]
 ...
 [0.220079   0.19796547 0.18914047 ... 0.21496978 0.23963419 0.16031656]
 [0.11527416 0.17701557 0.22045442 ... 0.21412304 0.22692391 0.24125946]
 [0.2624337  0.21422511 0.19079298 ... 0.21479893 0.22242686 0.23096368]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.14126131 0.14183712 0.15167326 ... 0.1333777  0.1390078  0.12878042]
 [0.11280769 0.16417676 0.10495213 ... 0.06761393 0.11782306 0.16335359]
 [0.10765615 0.15573108 0.12141576 ... 0.08981866 0.08262235 0.11504149]
 ...
 [0.19072717 0.22678399 0.19836363 ... 0.18889093 0.20612773 0.24982256]
 [0.10155988 0.15290698 0.20751551 ... 0.10908478 0.14274961 0.16090101]
 [0.17888135 0.19522029 0.19428757 ... 0.14328927 0.20752183 0.21588817]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.20993996 

tf.Tensor(
[[0.07412913 0.11139989 0.09327677 ... 0.07533097 0.1277901  0.08446133]
 [0.11810431 0.12763932 0.12796175 ... 0.10158831 0.12784562 0.13656303]
 [0.09723648 0.13551971 0.09021434 ... 0.11522111 0.10229477 0.11208606]
 ...
 [0.04254672 0.08374962 0.07202798 ... 0.07666352 0.09321037 0.1282847 ]
 [0.10229009 0.16918245 0.0975346  ... 0.11502257 0.13807395 0.13571408]
 [0.22476208 0.14906615 0.15388331 ... 0.10631064 0.1339455  0.1884695 ]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.13528398 0.13346988 0.12405229 ... 0.12595072 0.14962474 0.12055603]
 [0.1711342  0.17135733 0.19998214 ... 0.23029724 0.14693773 0.2070432 ]
 [0.16028756 0.1897487  0.15627494 ... 0.17892858 0.20102313 0.16016048]
 ...
 [0.12429738 0.18095249 0.14762935 ... 0.18014145 0.18550554 0.14713648]
 [0.13893846 0.10593554 0.08808869 ... 0.08565745 0.14967519 0.0957633 ]
 [0.17188668 0.14945    0.12906697 ... 0.13918114 0.17208105 0.13262078]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.13138282 

tf.Tensor(
[[0.09692031 0.10461494 0.1243718  ... 0.10144567 0.11052245 0.12413421]
 [0.1077472  0.12155765 0.15544865 ... 0.12918058 0.14967752 0.14183998]
 [0.07846227 0.12389648 0.08228782 ... 0.10888898 0.11068767 0.1375477 ]
 ...
 [0.10132524 0.14307177 0.09251073 ... 0.14281762 0.12495694 0.11247298]
 [0.08070433 0.09312207 0.08051547 ... 0.07052994 0.07284907 0.14062804]
 [0.13428035 0.09896907 0.12414676 ... 0.10816702 0.11399546 0.12516463]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.04590559 0.11919561 0.1032221  ... 0.10739809 0.06745991 0.11031863]
 [0.03991854 0.06336778 0.06172293 ... 0.05072325 0.04122871 0.05085349]
 [0.15028805 0.07998532 0.06100538 ... 0.09354353 0.09027189 0.06050599]
 ...
 [0.126329   0.14975485 0.12427181 ... 0.14040807 0.1806443  0.20588556]
 [0.07873145 0.13913661 0.15173155 ... 0.14331287 0.11605722 0.14694679]
 [0.22522482 0.21428248 0.17750344 ... 0.15690187 0.18258598 0.1865288 ]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.1198841  

...

tf.Tensor(
[[0.00879121 0.01242679 0.01212007 ... 0.01165348 0.01026389 0.00961909]
 [0.01898289 0.01563585 0.02536237 ... 0.0329611  0.01566669 0.0163044 ]
 [0.01858765 0.01497814 0.01465684 ... 0.02131006 0.01381376 0.02289522]
 ...
 [0.01811987 0.02303734 0.00808781 ... 0.01442981 0.01895732 0.01020283]
 [0.0037356  0.00283566 0.00441632 ... 0.00361249 0.00238785 0.00473234]
 [0.02752796 0.02247214 0.02476859 ... 0.02817976 0.02465448 0.03844985]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.00466004 0.00429001 0.00720987 ... 0.0083952  0.0053241  0.00476632]
 [0.02375627 0.00994059 0.00513095 ... 0.0107061  0.01725072 0.0084292 ]
 [0.01374295 0.00766101 0.00587875 ... 0.00714394 0.00772849 0.01230857]
 ...
 [0.01183316 0.00982869 0.01535639 ... 0.01883471 0.01685074 0.012445  ]
 [0.00767535 0.00818092 0.01129961 ... 0.00642335 0.01338008 0.00900164]
 [0.0084129  0.00865725 0.01250285 ... 0.01012814 0.00737631 0.0146018 ]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.01615426 

tf.Tensor(
[[0.00984806 0.01499006 0.00759003 ... 0.0220226  0.0101513  0.01082122]
 [0.01116592 0.00774628 0.00881955 ... 0.013044   0.01070952 0.01939452]
 [0.01178354 0.01603317 0.01826441 ... 0.00896177 0.01487276 0.01245695]
 ...
 [0.00920507 0.01105893 0.01250732 ... 0.02018917 0.016783   0.00967604]
 [0.00529632 0.00634316 0.00484541 ... 0.00424117 0.00883791 0.01012468]
 [0.00997591 0.02465767 0.0121347  ... 0.01498529 0.01795951 0.01607326]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.00920945 0.00831869 0.00408801 ... 0.00875315 0.00828144 0.00542459]
 [0.00599134 0.00803059 0.00932857 ... 0.0090633  0.00644365 0.00899488]
 [0.01384586 0.01251468 0.01610026 ... 0.01482579 0.01055631 0.01853451]
 ...
 [0.00618312 0.00554043 0.00649232 ... 0.00882894 0.00897896 0.00893927]
 [0.00245902 0.00170928 0.00198421 ... 0.0013907  0.00272509 0.00179765]
 [0.0122492  0.00685436 0.00556636 ... 0.01128575 0.01093557 0.00747693]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.01044184 

tf.Tensor(
[[0.01474693 0.02119541 0.00984144 ... 0.01393113 0.01841223 0.02275091]
 [0.0049926  0.00640398 0.00516289 ... 0.00673538 0.00567186 0.00601831]
 [0.00724283 0.01184392 0.01273638 ... 0.01310626 0.00918037 0.00858206]
 ...
 [0.00279868 0.00281325 0.00277022 ... 0.00406575 0.00334674 0.00398076]
 [0.01159814 0.01176625 0.01574552 ... 0.01253912 0.00727069 0.01669866]
 [0.00550395 0.00888899 0.00932389 ... 0.01078233 0.01149076 0.01303345]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.00791848 0.00720191 0.01133579 ... 0.00669596 0.00985956 0.00947735]
 [0.00254554 0.00347322 0.00609365 ... 0.00376663 0.00545001 0.00342184]
 [0.01331085 0.00837958 0.01253614 ... 0.01213896 0.01250628 0.00798082]
 ...
 [0.00537249 0.00956324 0.01354337 ... 0.0119659  0.00694066 0.00916782]
 [0.01638967 0.02382764 0.02244493 ... 0.01971704 0.02679405 0.0200817 ]
 [0.01217136 0.01693738 0.01705813 ... 0.01877642 0.01203844 0.01774567]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.01328039 

tf.Tensor(
[[0.01400027 0.02276516 0.02498576 ... 0.00958914 0.01767024 0.01892856]
 [0.02570939 0.02014846 0.02051994 ... 0.01764989 0.02980101 0.02227744]
 [0.00746343 0.00820652 0.00988087 ... 0.00633851 0.00913283 0.00888622]
 ...
 [0.00944886 0.00624931 0.01105645 ... 0.01623583 0.01168099 0.01006958]
 [0.01136622 0.01362136 0.01321301 ... 0.01791665 0.00817338 0.00935066]
 [0.01846907 0.01825836 0.02672386 ... 0.01691976 0.0193938  0.0288381 ]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.01669937 0.02603316 0.02022043 ... 0.01736975 0.01825583 0.02324086]
 [0.00872818 0.00922653 0.01126552 ... 0.00912037 0.00920323 0.01259652]
 [0.00399205 0.00276655 0.00288963 ... 0.00290364 0.00380394 0.00405058]
 ...
 [0.02855673 0.01850352 0.01078892 ... 0.0149667  0.01949012 0.01706487]
 [0.01133937 0.00775784 0.0086     ... 0.00371143 0.00791258 0.00740418]
 [0.00449896 0.00742522 0.01165688 ... 0.01208487 0.00895548 0.00941828]], shape=(64, 784), dtype=float32)
tf.Tensor(
[[0.00847319 

KeyboardInterrupt: 

In [65]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=2, batch_size=64)

Epoch 1/2
938/938 [==============================] - 2s 2ms/step - loss: 0.0748
Epoch 2/2
938/938 [==============================] - 1s 1ms/step - loss: 0.0676
